In [1]:
import numpy as np
import neal
import networkx as nx
from qiskit_aer import AerSimulator
from qiskit_algorithms import SamplingVQE
backend = AerSimulator(shots=1e4)
from docplex.mp.model import Model
import numpy as np
import random
import itertools
import random
import matplotlib.pyplot as plt
import networkx as nx
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit.library import TwoLocal
from qiskit import ClassicalRegister
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyMinimumEigensolver, NumPyEigensolver
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import time
from qiskit import QuantumCircuit
import neal
import random
import math
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization import QuadraticProgram
print(backend)

AerSimulator('aer_simulator')


In [111]:
n = 10
def generate_random_qubo_matrix(n, low=-1, high=1):
    Q = np.random.uniform(low, high, size=(n, n))
    Q = np.triu(Q) + np.triu(Q, 1).T
    return Q

qubo_matrix = generate_random_qubo_matrix(n)
print(qubo_matrix)

[[ 0.74864782 -0.10756736 -0.33828559  0.48255005  0.18479238  0.96499021
   0.84608014  0.58299601  0.28228911 -0.36719232]
 [-0.10756736 -0.68571827 -0.7205985   0.22144709  0.28699983 -0.7630373
   0.8136219  -0.05468644  0.26892347  0.56748088]
 [-0.33828559 -0.7205985  -0.77223128 -0.10705789 -0.7588589  -0.64942084
  -0.74587115 -0.42116186 -0.02772721 -0.54962748]
 [ 0.48255005  0.22144709 -0.10705789  0.44545295 -0.55235024  0.05517643
  -0.37055002 -0.21236738 -0.6327756  -0.62995622]
 [ 0.18479238  0.28699983 -0.7588589  -0.55235024  0.26175207 -0.25072012
  -0.56257892  0.52439285  0.11283326 -0.90434736]
 [ 0.96499021 -0.7630373  -0.64942084  0.05517643 -0.25072012 -0.57995783
   0.78940475  0.64867038 -0.12276458  0.30035959]
 [ 0.84608014  0.8136219  -0.74587115 -0.37055002 -0.56257892  0.78940475
  -0.2039105   0.27761284 -0.35494387 -0.91244341]
 [ 0.58299601 -0.05468644 -0.42116186 -0.21236738  0.52439285  0.64867038
   0.27761284  0.8696399  -0.51011771  0.15361375]
 

In [113]:
# Initialize the simulated annealer
sampler = neal.SimulatedAnnealingSampler()

# Run the solver
sampleset = sampler.sample_qubo(qubo_matrix, num_reads=1000)

# Print the best solution found
best_solution = sampleset.first.sample
best_energy = sampleset.first.energy
simulated_annealing_bit_string = []
for i in range(len(best_solution)):
    simulated_annealing_bit_string.append(best_solution[i])

print("Best Solution:", simulated_annealing_bit_string)
print("Best Energy:", best_energy)

Best Solution: [0, 0, 1, 1, 1, 1, 1, 0, 1, 1]
Best Energy: -14.64323051385512


In [115]:
def matrix_to_qubo_dict(matrix):
    Q = {}
    n = matrix.shape[0]
    for i in range(n):
        for j in range(i, n):  # Only upper triangle (QUBO is symmetric)
            Q[(i, j)] = matrix[i, j]
    return Q

Q = matrix_to_qubo_dict(qubo_matrix)

def qubo_dict_to_matrix(Q):
    # Get the size of the matrix
    n = max(max(i, j) for i, j in Q.keys()) + 1
    matrix = np.zeros((n, n))

    for (i, j), w in Q.items():
        matrix[i, j] = w
        if i != j:
            matrix[j, i] = w  # Ensure symmetry

    return matrix

In [117]:
def qubo_to_graph(Q):
    """ Converts a QUBO dictionary into a NetworkX graph. """
    G = nx.Graph()
    for (i, j), w in Q.items():
        if i == j:
            G.add_node(i, weight=w)
        else:
            G.add_edge(i, j, weight=w)
    return G

In [119]:
def partition_qubo(Q, num_parts):
    """ Partitions a QUBO graph into smaller sub-QUBOs. """
    G = qubo_to_graph(Q)
    partition = nx.community.kernighan_lin_bisection(G)  # Bisection partitioning
    sub_QUBOs = []

    for part in partition:
        sub_Q = {(i, j): Q[i, j] for i in part for j in part if (i, j) in Q}
        sub_QUBOs.append(sub_Q)
    
    return sub_QUBOs, partition

In [121]:
def qubo_to_hamiltonian(Q):
    n = Q.shape[0]
    paulis = []
    coeffs = []
    offset = 0.0

    for i in range(n):
        offset += Q[i, i] / 4
        coeff = -Q[i, i] / 2
        if coeff != 0.0:
            z_str = ['I'] * n
            z_str[i] = 'Z'
            paulis.append("".join(z_str))
            coeffs.append(coeff)

        for j in range(i+1, n):
            if Q[i, j] != 0.0:
                offset += Q[i, j] / 4
                coeff_zz = Q[i, j] / 4
                coeff_z1 = -Q[i, j] / 4
                coeff_z2 = -Q[i, j] / 4

                z_str = ['I'] * n
                z_str[i] = 'Z'
                z_str[j] = 'Z'
                paulis.append("".join(z_str))
                coeffs.append(coeff_zz)

                z_str1 = ['I'] * n
                z_str1[i] = 'Z'
                paulis.append("".join(z_str1))
                coeffs.append(coeff_z1)

                z_str2 = ['I'] * n
                z_str2[j] = 'Z'
                paulis.append("".join(z_str2))
                coeffs.append(coeff_z2)

    # Create the SparsePauliOp
    hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

    return hamiltonian, offset

In [123]:
def cost_func(params, ansatz, hamiltonian, estimator):
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]
#    cost_history_dict["iters"] += 1
#    cost_history_dict["prev_vector"] = params
#    cost_history_dict["cost_history"].append(energy)
    return energy
def solve_qubo(Q):
    qubo_matrix = qubo_dict_to_matrix(Q)
    hamiltonian = qubo_to_hamiltonian(qubo_matrix)[0]
    ansatz = EfficientSU2(hamiltonian.num_qubits)
    ansatz.measure_all()
    num_params = ansatz.num_parameters
    target = backend.target
    pm = generate_preset_pass_manager(target=target, optimization_level=3)
    ansatz_isa = pm.run(ansatz)
    hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)
    cost_history_dict = {
        "prev_vector": None,
        "iters": 0,
        "cost_history": [],
    }
    x0 = 2 * np.pi * np.random.random(num_params)
    with Session(backend=backend) as session:
        estimator = Estimator(mode=session)
        estimator.options.default_shots = 10000
    
        res = minimize(
            cost_func,
            x0,
            args=(ansatz_isa, hamiltonian_isa, estimator),
            method="cobyla",
             options={ # Optional: prints optimization progress
                 "rhobeg": 2}
        )
    optimal_params = res.x
    optimized_circuit = ansatz_isa.assign_parameters(res.x)
    sampler = AerSimulator()
    job = sampler.run([optimized_circuit])
    result = job.result()
    counts_bin = result.get_counts()
    most_probable = max(counts_bin, key=counts_bin.get)
    state_as_list = [int(bit) for bit in most_probable]
#    state_as_list.reverse()
    print(state_as_list)
    return state_as_list, 0

In [125]:
def refine_solution(Q, initial_solution):
    """ Refining the combined solution by solving a reduced full QUBO. """
    sampler = neal.SimulatedAnnealingSampler()
    
    # Convert the initial solution into a partial assignment
    fixed_vars = {i: v for i, v in initial_solution.items()}

    # Solve the full QUBO again with this partial assignment
    sampleset = sampler.sample_qubo(Q, num_reads=100)

    return sampleset.first.sample, sampleset.first.energy

In [127]:
sub_QUBOs, partitions = partition_qubo(Q, num_parts=5) 
print(sub_QUBOs[0])

{(1, 1): -0.6857182681518326, (1, 2): -0.7205985032457614, (1, 3): 0.2214470865429885, (1, 8): 0.26892346519584676, (1, 9): 0.5674808819529573, (2, 2): -0.772231278501208, (2, 3): -0.10705788585169507, (2, 8): -0.027727214050056626, (2, 9): -0.5496274801672907, (3, 3): 0.44545294990857354, (3, 8): -0.6327755976408449, (3, 9): -0.6299562200005564, (8, 8): 0.030275260863112496, (8, 9): -0.007976182519481734, (9, 9): -0.060219256526671705}


In [ ]:
sub_solutions = [solve_qubo(sub_Q) for sub_Q in sub_QUBOs]

merged_solution = {}
for part_bits, part_vars in zip(sub_solutions, partitions):
    for var, bit in zip(part_vars, part_bits):
        merged_solution[var] = bit

final_solution, final_energy = refine_solution(Q, merged_solution)

final_bit_string = [final_solution[i] for i in sorted(final_solution)]

print("Refined Final Solution:", final_bit_string)
print("Refined Final Energy:", final_energy)

In [ ]:
if final_solution == best_solution:
    print("Both solutions are the same and correct.")
else:
    print("Both solutions are different and incorrect.")